In [ ]:
import tkinter as tk
from tkinter import messagebox, filedialog
import json

class SimpleApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Student Information App")

        # Labels and Entry fields
        tk.Label(root, text="Name:").grid(row=0, column=0, padx=10, pady=5)
        tk.Label(root, text="Age:").grid(row=1, column=0, padx=10, pady=5)
        tk.Label(root, text="Grade:").grid(row=2, column=0, padx=10, pady=5)

        self.name_var = tk.StringVar()
        self.age_var = tk.StringVar()
        self.grade_var = tk.StringVar()

        tk.Entry(root, textvariable=self.name_var).grid(row=0, column=1)
        tk.Entry(root, textvariable=self.age_var).grid(row=1, column=1)
        tk.Entry(root, textvariable=self.grade_var).grid(row=2, column=1)

        # Buttons
        tk.Button(root, text="Save", command=self.save_data).grid(row=3, column=0, pady=10)
        tk.Button(root, text="Load", command=self.load_data).grid(row=3, column=1, pady=10)

    # --- Core functionality ---
    def save_data(self):
        name = self.name_var.get()
        age = self.age_var.get()
        grade = self.grade_var.get()

        # Input validation
        if not name or not age or not grade:
            messagebox.showerror("Error", "All fields are required!")
            return
        if not age.isdigit():
            messagebox.showerror("Error", "Age must be a number!")
            return

        data = {"name": name, "age": int(age), "grade": grade}

        with open("students.json", "w") as f:
            json.dump(data, f)

        messagebox.showinfo("Success", "Data saved successfully!")

    def load_data(self):
        try:
            with open("students.json", "r") as f:
                data = json.load(f)
            self.name_var.set(data["name"])
            self.age_var.set(data["age"])
            self.grade_var.set(data["grade"])
        except FileNotFoundError:
            messagebox.showerror("Error", "No saved data found!")

if __name__ == "__main__":
    root = tk.Tk()
    app = SimpleApp(root)
    root.mainloop()


In [ ]:
django-admin startproject school_project
cd school_project
python manage.py startapp students


In [ ]:
python manage.py makemigrations
python manage.py migrate


In [ ]:
from django.shortcuts import render, redirect, get_object_or_404
from .models import Student

def student_list(request):
    students = Student.objects.all()
    return render(request, "students/student_list.html", {"students": students})

def add_student(request):
    if request.method == "POST":
        name = request.POST["name"]
        age = request.POST["age"]
        grade = request.POST["grade"]
        Student.objects.create(name=name, age=age, grade=grade)
        return redirect("student_list")
    return render(request, "students/add_student.html")

def delete_student(request, id):
    student = get_object_or_404(Student, id=id)
    student.delete()
    return redirect("student_list")


In [ ]:
from django.urls import path
from . import views

urlpatterns = [
    path('', views.student_list, name='student_list'),
    path('add/', views.add_student, name='add_student'),
    path('delete/<int:id>/', views.delete_student, name='delete_student'),
]


In [ ]:
from django.contrib import admin
from django.urls import path, include

urlpatterns = [
    path('admin/', admin.site.urls),
    path('', include('students.urls')),
]


In [ ]:
<h1>Student List</h1>
<a href="{% url 'add_student' %}">Add Student</a>
<ul>
  {% for student in students %}
  <li>{{ student.name }} ({{ student.age }}) - {{ student.grade }}
    <a href="{% url 'delete_student' student.id %}">Delete</a>
  </li>
  {% endfor %}
</ul>


In [ ]:
python manage.py createsuperuser


In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox, filedialog
import json
import csv
import os

class StudentApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Student Management System")
        self.students = []  # store all student records

        # --- Input Section ---
        frame_input = tk.LabelFrame(root, text="Student Details", padx=10, pady=10)
        frame_input.pack(fill="x", padx=10, pady=5)

        tk.Label(frame_input, text="Name:").grid(row=0, column=0, padx=5, pady=5)
        tk.Label(frame_input, text="Age:").grid(row=0, column=2, padx=5, pady=5)
        tk.Label(frame_input, text="Grade:").grid(row=0, column=4, padx=5, pady=5)

        self.name_var = tk.StringVar()
        self.age_var = tk.StringVar()
        self.grade_var = tk.StringVar()

        tk.Entry(frame_input, textvariable=self.name_var, width=20).grid(row=0, column=1)
        tk.Entry(frame_input, textvariable=self.age_var, width=10).grid(row=0, column=3)
        tk.Entry(frame_input, textvariable=self.grade_var, width=10).grid(row=0, column=5)

        tk.Button(frame_input, text="Add Student", command=self.add_student).grid(row=0, column=6, padx=5)

        # --- Search Section ---
        frame_search = tk.Frame(root)
        frame_search.pack(fill="x", padx=10, pady=5)

        tk.Label(frame_search, text="Search by Name:").pack(side="left", padx=5)
        self.search_var = tk.StringVar()
        tk.Entry(frame_search, textvariable=self.search_var, width=20).pack(side="left")
        tk.Button(frame_search, text="Search", command=self.search_student).pack(side="left", padx=5)
        tk.Button(frame_search, text="Show All", command=self.display_students).pack(side="left")

        # --- Table Section ---
        frame_table = tk.Frame(root)
        frame_table.pack(fill="both", expand=True, padx=10, pady=5)

        columns = ("Name", "Age", "Grade")
        self.tree = ttk.Treeview(frame_table, columns=columns, show="headings")
        for col in columns:
            self.tree.heading(col, text=col)
            self.tree.column(col, width=100, anchor="center")
        self.tree.pack(fill="both", expand=True)

        # --- Buttons ---
        frame_buttons = tk.Frame(root)
        frame_buttons.pack(fill="x", padx=10, pady=10)

        tk.Button(frame_buttons, text="Save Data", command=self.save_data).pack(side="left", padx=5)
        tk.Button(frame_buttons, text="Load Data", command=self.load_data).pack(side="left", padx=5)
        tk.Button(frame_buttons, text="Export CSV", command=self.export_csv).pack(side="left", padx=5)

    # --- Core Functions ---
    def add_student(self):
        name = self.name_var.get().strip()
        age = self.age_var.get().strip()
        grade = self.grade_var.get().strip()

        if not name or not age or not grade:
            messagebox.showerror("Error", "All fields are required!")
            return
        if not age.isdigit():
            messagebox.showerror("Error", "Age must be a number!")
            return

        student = {"name": name, "age": int(age), "grade": grade}
        self.students.append(student)
        self.display_students()
        self.clear_inputs()

    def display_students(self, students=None):
        # Clear the table first
        for item in self.tree.get_children():
            self.tree.delete(item)
        if students is None:
            students = self.students
        for s in students:
            self.tree.insert("", "end", values=(s["name"], s["age"], s["grade"]))

    def clear_inputs(self):
        self.name_var.set("")
        self.age_var.set("")
        self.grade_var.set("")

    def search_student(self):
        query = self.search_var.get().lower()
        if not query:
            messagebox.showinfo("Info", "Enter a name to search.")
            return
        results = [s for s in self.students if query in s["name"].lower()]
        if results:
            self.display_students(results)
        else:
            messagebox.showinfo("Search", "No matching students found.")

    def save_data(self):
        with open("students.json", "w") as f:
            json.dump(self.students, f)
        messagebox.showinfo("Success", "Data saved to students.json")

    def load_data(self):
        if not os.path.exists("students.json"):
            messagebox.showerror("Error", "No saved file found!")
            return
        with open("students.json", "r") as f:
            self.students = json.load(f)
        self.display_students()
        messagebox.showinfo("Loaded", "Data loaded successfully!")

    def export_csv(self):
        if not self.students:
            messagebox.showerror("Error", "No data to export!")
            return
        file_path = filedialog.asksaveasfilename(defaultextension=".csv",
                                                 filetypes=[("CSV files", "*.csv")])
        if file_path:
            with open(file_path, "w", newline="") as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=["name", "age", "grade"])
                writer.writeheader()
                writer.writerows(self.students)
            messagebox.showinfo("Exported", f"Data exported to {file_path}")

if __name__ == "__main__":
    root = tk.Tk()
    app = StudentApp(root)
    root.mainloop()


In [ ]:
from django.contrib.auth import authenticate, login, logout
from django.contrib.auth.models import User
from django.contrib.auth.decorators import login_required

def register_view(request):
    if request.method == "POST":
        username = request.POST["username"]
        password = request.POST["password"]
        if User.objects.filter(username=username).exists():
            return render(request, "students/register.html", {"error": "Username already exists"})
        User.objects.create_user(username=username, password=password)
        return redirect("login")
    return render(request, "students/register.html")

def login_view(request):
    if request.method == "POST":
        username = request.POST["username"]
        password = request.POST["password"]
        user = authenticate(request, username=username, password=password)
        if user:
            login(request, user)
            return redirect("dashboard")
        else:
            return render(request, "students/login.html", {"error": "Invalid credentials"})
    return render(request, "students/login.html")

def logout_view(request):
    logout(request)
    return redirect("login")


In [ ]:
urlpatterns = [
    path("login/", views.login_view, name="login"),
    path("logout/", views.logout_view, name="logout"),
    path("register/", views.register_view, name="register"),
    path("dashboard/", views.dashboard_view, name="dashboard"),
    # ... other student CRUD paths
]


In [ ]:
@login_required
def dashboard_view(request):
    students = Student.objects.all()
    total = students.count()
    avg_age = students.aggregate(models.Avg("age"))["age__avg"]
    return render(request, "students/dashboard.html", {
        "total": total,
        "avg_age": avg_age or 0,
        "students": students
    })


In [ ]:
<h1>Dashboard</h1>
<p>Total Students: {{ total }}</p>
<p>Average Age: {{ avg_age|floatformat:1 }}</p>

<table border="1">
  <tr><th>Name</th><th>Age</th><th>Grade</th></tr>
  {% for s in students %}
  <tr>
    <td>{{ s.name }}</td>
    <td>{{ s.age }}</td>
    <td>{{ s.grade }}</td>
  </tr>
  {% endfor %}
</table>

<a href="{% url 'logout' %}">Logout</a>
